In [ ]:
import os
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
import sys
sys.path.append("saia-finetuning")
from datetime import datetime, timezone
from functools import partial
from typing import Any

from experiments.investigation_reports.tools import extract_splunk_system_data_reports, extract_data_reports, response_formatter_reports, finding_fields_extractor, notes_fields_extractor
from tooling.llm_engine import GenerativeModel

from tooling.agents.executor import Executor
from tooling.agents.state import StateGraph
# from tooling.llm_engine.llama3_1Instruct import Llama3_1Instruct as llm
from tooling.llm_engine.azure_oai import AzureGPT as llm
from experiments.investigation_reports.constants import FINDINGS_PROMPT, FINDINGS_TEMPLATE, NOTES_TEMPLATE, NOTES_PROMPT
import pandas as pd
from io import StringIO



In [ ]:
# Construct state graph
import pandas as pd
action_space = StateGraph()

# Instantiate LLM
model = llm()


action_space.add_action(action_name="data_chunker", callable=extract_splunk_system_data_reports)
action_space.add_action(
    action_name="information_extractor",
    callable=extract_data_reports
)

action_space.add_action(
    action_name="finding_fields_extractor",
    callable=partial(finding_fields_extractor, model=model)
)
action_space.add_action(
    action_name="notes_fields_extractor",
    callable=partial(notes_fields_extractor, model=model)
)

action_space.add_action(
    action_name="response_formatter",
    callable=response_formatter_reports
)

action_space.add_composition(
    action_name_0="data_chunker",
    action_name_1="information_extractor",
)
action_space.add_composition(
    action_name_0="data_chunker",
    action_name_1="finding_fields_extractor",
)

action_space.add_composition(
    action_name_0="data_chunker",
    action_name_1="notes_fields_extractor",
)

action_space.add_composition(
    action_name_0="notes_fields_extractor",
    action_name_1="response_formatter",
)

def state_contains_keys(keys:list[str], context:dict[str,Any]) -> bool:
    return set(context).issuperset(keys)



action_space.add_composition(
    action_name_0="finding_fields_extractor",
    action_name_1="response_formatter",
    condition=partial(state_contains_keys, keys=["extract_system_data","findings_response_text", "notes_response_text"])
)

action_space.add_composition(
    action_name_0="notes_fields_extractor",
    action_name_1="response_formatter",
    condition=partial(state_contains_keys, keys=["extract_system_data","findings_response_text", "notes_response_text"])
)

action_space.add_composition(
    action_name_0="information_extractor",
    action_name_1="response_formatter",
    condition=partial(state_contains_keys, keys=["extract_system_data","findings_response_text", "notes_response_text"])
)


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

graph = action_space.graph
pos = nx.planar_layout(action_space.graph)
nx.draw_networkx_nodes(graph, pos)
nx.draw_networkx_edges(graph, pos)
nx.draw_networkx_labels(graph, pos, font_size=8)
plt.show()

In [ ]:
# load data
import json

path3='experiments/investigation_reports/data/July25_10_Investigations_Analyst_Notes.json'

data_all3=[]
with open(path3) as f:
    data_all3 = json.load(f)
print(data_all3[0])
investigations=data_all3

In [ ]:
# instantiate executor
executor = Executor()
executor.add_state_graph(action_space)

# construct initial state if applicable
INVESTIGATION = investigations[0]
state = {
    "investigation_data": INVESTIGATION
}

executor.augment_state(state)
executor.set_entry_point(action_name="data_chunker")

# run executor
terminal_state = executor.run()